In [ ]:
!pip install contractions
!pip install emoji
!pip install ekphrasis
!pip install -U -q PyDrive
!pip install yfinance --upgrade --no-cache-dir

import pandas as pd
import re
import emoji
import nltk
import contractions
import torch

nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer 
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize 
from nltk.tokenize.treebank import TreebankWordDetokenizer
import requests 
from pprint import pprint
import numpy as np
from nltk.stem import PorterStemmer

import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras

import os
import pprint
import json
import random
import string
import sys
from ekphrasis.classes.segmenter import Segmenter
import itertools


from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

from google.colab import drive

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter

import datetime
from datetime import datetime, timedelta, date

import yfinance as yf
from datetime import datetime, timedelta, date

#mount's to Google Drive
drive.mount('/content/drive')

#connect's to Google Cloud SDK
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
df = pd.read_csv('/content/combined_0.csv')
df = df.sample(frac=1)
df.drop(df[df['Date'] <= '2019-07-20'].index, inplace = True)
df['label'].value_counts()

1    27545
0    25205
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['message'].values, df['label'].values, test_size=0.1, random_state=42)

In [ ]:
df_train = pd.DataFrame(y_train)
df_train.value_counts()

1    24836
0    22639
dtype: int64

In [ ]:
df_test = pd.DataFrame(y_test)
df_test.value_counts()

1    2709
0    2566
dtype: int64

In [ ]:
sentences = X_train
labels = y_train

**BERT**

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
input_ids = []
attention_masks = []

# For each sentence encode_plus would tokenize, add special tokens ([CLS], [SEP]), map tokens to IDs, create attention masks
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        str(sent),                     
                        add_special_tokens = True,
                        max_length = 160,         
                        truncation = True,
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',    
                   )   
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

In [ ]:
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Original:  pg filed sec form 4 insider keith r alexandra sold 7627 common stock price 120 4665 2019 08 27
Token IDs: tensor([  101, 18720,  6406, 10819,  2433,  1018, 25297,  6766,  1054, 10481,
         2853,  6146, 22907,  2691,  4518,  3976,  6036,  4805, 26187, 10476,
         5511,  2676,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,    

In [ ]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)

# Creating TensorDataset by combining the training inputs
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Dividing the training and validation dataset by selecting samples randomly
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{} training samples'.format(train_size))
print('{} validation samples'.format(val_size))

42727 training samples
4748 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = batch_size 
        )

validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), 
            batch_size = batch_size 
        )

In [ ]:
from transformers import BertModel, BertForSequenceClassification, AdamW, BertConfig


# Loading BertForSequenceClassification model for training
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 2, #change here if classes changed to 3   
    output_attentions = False,
    output_hidden_states = False, 
)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
torch.cuda.empty_cache()

# Assigning the model on the GPU.
model.cuda()